# Projet 5 : Segmenter des clients d'un site e-commerce

*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours *Data Scientist* d'OpenClassroooms.

L'objectif pricipal est de réaliser **une segmentation des clients** d'un site de e-commerce, **une proposition de contrat de maintenance** devra être inclue.

Les données mises à notre disposition proviennent du site *kaggle* :
https://www.kaggle.com/olistbr/brazilian-ecommerce

# Partie I : Data Wrangling

L'objectif de ce notebook est de décrire les opérations de nettoyage nécessaires à l'obtention d'un jeu de données exploitable.

In [1]:
# Import usual libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns

In [2]:
# Change some default parameters of matplotlib using seaborn
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Récupération-des-données" data-toc-modified-id="Récupération-des-données-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Récupération des données</a></span></li><li><span><a href="#Construction-d'une-table-unique" data-toc-modified-id="Construction-d'une-table-unique-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Construction d'une table unique</a></span></li><li><span><a href="#Ingénierie-des-variables" data-toc-modified-id="Ingénierie-des-variables-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ingénierie des variables</a></span><ul class="toc-item"><li><span><a href="#Variables-liées-aux-transactions" data-toc-modified-id="Variables-liées-aux-transactions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Variables liées aux transactions</a></span></li><li><span><a href="#Variables-liées-aux-produits" data-toc-modified-id="Variables-liées-aux-produits-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Variables liées aux produits</a></span></li></ul></li></ul></div>

## Récupération des données

Une fois les données téléchargées, nous pouvons les charger dans un DataFrame en utilisant la librairie **pandas**.

In [3]:
df_customers =  pd.read_csv("data/olist_customers_dataset.csv")
df_geolocation = pd.read_csv("data/olist_geolocation_dataset.csv")
df_order_items = pd.read_csv("data/olist_order_items_dataset.csv")
df_order_payments = pd.read_csv("data/olist_order_payments_dataset.csv")
df_order_reviews = pd.read_csv("data/olist_order_reviews_dataset.csv")
df_orders = pd.read_csv("data/olist_orders_dataset.csv")
df_products = pd.read_csv("data/olist_products_dataset.csv")
df_sellers = pd.read_csv("data/olist_sellers_dataset.csv")
df_translation = pd.read_csv("data/product_category_name_translation.csv")

Regardons ce qui est contenu dans chaque DataFrame.

In [4]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [5]:
df_geolocation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


In [6]:
df_order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [7]:
df_order_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [8]:
df_order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   review_id                100000 non-null  object
 1   order_id                 100000 non-null  object
 2   review_score             100000 non-null  int64 
 3   review_comment_title     11715 non-null   object
 4   review_comment_message   41753 non-null   object
 5   review_creation_date     100000 non-null  object
 6   review_answer_timestamp  100000 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [9]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [10]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [11]:
df_sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [12]:
df_translation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


Nous pouvons noter plusieurs choses :
1. Travailler avec huit tables n'est pas pratique, il serait plus de les regrouper en une seule
2. La table geolocation n'apporte rien de plus que la table *customers*
3. Beaucoup de données redondantes
4. Les tables contiennent très peu de données manquantes

Contruisons une table unique regroupant toutes les données.

## Construction d'une table unique

In [35]:
from functools import reduce

list_df_orders = [df_orders,
                  df_order_items,
                  df_order_payments,
                  df_order_reviews]
# Merge all DataFrame handling orders data
data = reduce(lambda left, right: pd.merge(left, right, how="outer",
                                           on="order_id", copy=True),
              list_df_orders)
# Add customers data
data = data.merge(right=df_customers, how="outer",
                  on="customer_id", copy=False)
# Add products data
data = data.merge(right=df_products, how="outer",
                  on="product_id", copy=False)
# Add the English translation of the product category
data = data.merge(right=df_translation, how="outer",
                  on="product_category_name", copy=False)
# Add sellers data
data = data.merge(right=df_sellers, how="outer",
                  on="seller_id", copy=False)

## Ingénierie des variables

Bon nombre des variables ne sont pas exploitables en l'état, il va falloir les transformer avant de penser à les utiliser pour faire tourner des modèles de partionnement.

De plus, ayant pour but de faire une segmentation des clients il va nous falloir regrouper les données par client. Dans la base de donnée, un client unique est assigné à chaque transaction. Ainsi, un client ayant effectué plusieurs transactions se verra attribuer plusieurs *customer_id*. C'est pourquoi nous avons aussi accès à la variable *customer_unique_id*. C'est cette dernière que nous utiliserons pour regrouper les données.

### Variables liées aux transactions

Nous commencerons par traités les variables de transaction. Pour chaque client, nous calculerons les variables suivantes :
- Nombre de transactions effectuées
- Nombre d'articles du panier moyen
- Prix du panier moyen
- Frais de port du panier moyen

In [39]:
values = ["order_id",
          "product_id",
          "price",
          "freight_value"]
aggfunc = {"order_id": pd.Series.nunique,
           "product_id": "count",
           "price": sum,
           "freight_value": sum}

pivot_orders = (pd.pivot_table(data, index="customer_unique_id",
                              values=values, aggfunc=aggfunc)
                  .rename(columns={"order_id": "n_orders",
                                   "product_id": "n_products_order",
                                   "price": "price_order",
                                   "freight_value": "freight_value_order"})
                  .assign(n_products=lambda df: df["n_products_order"]/df["n_orders"],
                          price=lambda df: df["price_order"]/df["n_orders"],
                          freight_value= lambda df: df["freight_value_order"]/df["n_orders"]))

### Variables liées aux produits

Nous traiterons ensuite les variables liées aux produits. Pour chaque client, nous calculerons les variables suivantes :
- prix moyen par produit
- nombre moyen de photos par produit
- longeur moyenne de description par produit
- frais de port moyens par produit

In [20]:
df_customers["customer_state"].value_counts()

SP    41746
RJ    12852
MG    11635
RS     5466
PR     5045
SC     3637
BA     3380
DF     2140
ES     2033
GO     2020
PE     1652
CE     1336
PA      975
MT      907
MA      747
MS      715
PB      536
PI      495
RN      485
AL      413
SE      350
TO      280
RO      253
AM      148
AC       81
AP       68
RR       46
Name: customer_state, dtype: int64